In [1]:
import sys
import os
import datetime as dt
from pathlib import Path
from collections import OrderedDict
import math
from shutil import copyfile
import json

import requests
import gtfstk as gt
from google.protobuf import json_format
from google.transit import gtfs_realtime_pb2

DIR = Path('..')
sys.path.append(str(DIR))

%load_ext autoreload
%autoreload 2

import gtfsrtk as gr

DATA_DIR = DIR/'data'

In [35]:
path = DATA_DIR/'tripUpdates.pb'
feed = gr.read_gtfsr(path)

path = DATA_DIR/'tripUpdates.json'
feed = gr.write_gtfsr(feed, path, to_json=True)


In [31]:
feed = gtfs_realtime_pb2.FeedMessage()
feed.ParseFromString(b'')
gr.get_timestamp(feed)

'19700101120000'

In [ ]:
# Get a recent Auckland GTFSR trip update feed
api_key = os.environ['AT_KEY']
url = 'https://api.at.govt.nz/v2/public/realtime/?'#tripupdates?'
headers = {
  'Ocp-Apim-Subscription-Key': api_key,
  }
params = {
  'callback': '',
  'tripid': '',
  'vehicleid': '',
  }    

r = requests.get(url, headers=headers, params=params)
feed = r.json()['response']


In [38]:

# Fix Auckland feeds on file



def fix_auckland_gtfsr(feed):
    """
    Given an Auckland GTFSR JSON feed (dict),
    reformat it to match the GTFSR.
    In particular, convert each 'time' and 'timestamp' field values from float
    to integer (POSIX time) and convert each 'stop_time_update' field
    value from a dictionary to a list containing that dictionary.
    Return the resulting feed (dict).
    """
    s = feed
    t = {}
    if isinstance(s, dict):      
        for k, v in s.items():
            if isinstance(v, dict):
                if k == 'stop_time_update':
                    t[k] = [fix_auckland_gtfsr(v)]
                else:
                    t[k] = fix_auckland_gtfsr(v)
            elif isinstance(v, list):
                t[k] = [fix_auckland_gtfsr(x) for x in v]
            elif k in ['timestamp', 'time']:
                t[k] = int(v)
            else:
                t[k] = v 
    elif isinstance(s, list):
        t = [fix_auckland_gtfsr(x) for x in s]
    else:
        t = s
    return t


for path in (DATA_DIR/'test_gtfsr').iterdir():
    if '_t' in path.name:
        path.unlink()
        continue
        
    with path.open() as src:
        feed = json.load(src)
    
    feed = fix_auckland_gtfsr(feed)
    with path.open('w') as tgt:
        json.dump(feed, tgt, indent=2)



In [37]:
path = DATA_DIR/'test_gtfsr'/'20160519080120_t.json'

feed  = gr.read_gtfsr(path, from_json=True)
feed

header {
  gtfs_realtime_version: "1.0"
  incrementality: FULL_DATASET
  timestamp: 1463601680
}
entity {
  id: "303f3d92-5103-5850-a6c7-ef5af1e983a6"
  is_deleted: false
  trip_update {
    trip {
      trip_id: "15455002274-20160512154122_v40.34"
      schedule_relationship: SCHEDULED
      route_id: "45502-20160512154122_v40.34"
    }
    stop_time_update {
      stop_sequence: 20
      arrival {
        delay: -3
        time: 1463601670
      }
      stop_id: "2312"
      schedule_relationship: SCHEDULED
    }
    vehicle {
      id: "2916"
    }
    timestamp: 1463601670
  }
}
entity {
  id: "c004a9f0-9dd0-550f-e49d-82354bec4698"
  is_deleted: false
  trip_update {
    trip {
      trip_id: "15348001791-20160512154122_v40.34"
      schedule_relationship: SCHEDULED
      route_id: "34804-20160512154122_v40.34"
    }
    stop_time_update {
      stop_sequence: 44
      departure {
        delay: 834
        time: 1463601659
      }
      stop_id: "6925"
      schedule_relationship:

In [ ]:
gr.extract_delays(feed)

In [ ]:
# Create augmented stop times

gtfsr_dir = DATA_DIR/'test_gtfsr'
gtfsr_feeds = []
for f in gtfsr_dir.iterdir():
    with f.open() as src:
        gtfsr_feeds.append(json.load(src))
gtfs_feed = gt.read_gtfs(DATA_DIR/'test_gtfs.zip', dist_units_in='km') # Good from 20160519
date = '20160519'

ast = build_augmented_stop_times(gtfsr_feeds, gtfs_feed, date)

tid = '14005028723-20160512154122_v40.34'
ast[ast['trip_id'] == tid]

In [ ]:
# Fill null delays

f = interpolate_delays(ast, dist_threshold=1)
f[f['trip_id'] == tid]
